# Setup

Ensure all dependencies are installed.


In [1]:
%pip install -U -r requirements.txt

  Cloning ssh://****@gitlab.utwente.nl/s2219735-bsc-research-project/mesh-to-points.git (to revision main) to c:\users\julia\appdata\local\temp\pip-install-2_5_z2pa\m2p_b8df54f727504706812e3f62fdc062d9
  Resolved ssh://****@gitlab.utwente.nl/s2219735-bsc-research-project/mesh-to-points.git to commit 78899e62db635567c95b5e83565a036f4974d071
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet 'ssh://****@gitlab.utwente.nl/s2219735-bsc-research-project/mesh-to-points.git' 'C:\Users\julia\AppData\Local\Temp\pip-install-2_5_z2pa\m2p_b8df54f727504706812e3f62fdc062d9'


Check if we have GPU support, and if not, warn the user.


In [2]:
import torch
import warnings

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    warnings.warn("CUDA is not available. Running on CPU.")

C:\Users\julia\AppData\Local\Temp\ipykernel_512\591437561.py:7: UserWarning: CUDA is not available. Running on CPU.
  warnings.warn("CUDA is not available. Running on CPU.")


Load our dataset...


In [1]:
import os
from models.data_sources import ModelNet40
from models.dataset import PointCloudDataset
from typing import List

train_folder = "data/train"
test_folder = "data/test"

train_dataset: PointCloudDataset = None
test_dataset: PointCloudDataset = None

if os.path.exists(train_folder) and os.path.exists(test_folder):
    print("Train and test data found. Reading data...")
    train_files: List[str] = []
    test_files: List[str] = []

    for root, dirs, files in os.walk(train_folder):
        for file in files:
            train_files.append(os.path.join(root, file))
    
    for root, dirs, files in os.walk(test_folder):
        for file in files:
            test_files.append(os.path.join(root, file))

    train_dataset = PointCloudDataset(train_files)
    test_dataset = PointCloudDataset(test_files)
else:
    train_dataset, test_dataset = ModelNet40.download(npoints=2500, train_outdir=train_folder, test_outdir=test_folder)

print(
    f"Training data: {len(train_dataset)} samples with {len(train_dataset.classes)} classes"
)
print(
    f"Test data size: {len(test_dataset)} samples with {len(test_dataset.classes)} classes"
)

ModuleNotFoundError: No module named 'dataset'

# Training

Train a new classifier or load a new pre-trained one:


In [4]:
from pointnet.train_classification import train_classification
from pointnet.model import PointNetCls

classifier_path = ""  # pointnet/classifier.pth

if classifier_path and os.path.exists(classifier_path):
    print(f"Loading existing model from {classifier_path}...")
    classifier = PointNetCls(k=len(train_dataset.classes))
    classifier.load_state_dict(torch.load(classifier_path, weights_only=True))
    print("Classifier loaded successfully.")
else:
    classifier = train_classification(
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        epochs=25,
        outf="./pointnet/cls",
    )

Random Seed:  7713


OutOfMemoryError: CUDA out of memory. Tried to allocate 314.00 MiB. GPU 0 has a total capacity of 14.57 GiB of which 12.75 MiB is free. Process 3079325 has 3.71 GiB memory in use. Process 3146416 has 2.06 GiB memory in use. Process 3171325 has 6.14 GiB memory in use. Process 3146413 has 1.40 GiB memory in use. Process 3245348 has 1.25 GiB memory in use. Of the allocated memory 880.33 MiB is allocated by PyTorch, and 267.67 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Run a simple classification to get a visualization of our classifier.


In [ ]:
import random
import torch
from utils.plot import plot_points

i = random.randint(0, len(test_dataset) - 1)

input_data = test_dataset[i][0].unsqueeze(0)
input_data = input_data.transpose(1, 2)

classifier.eval()

with torch.no_grad():
    output = classifier(input_data)


scores: torch.Tensor = output[0][0]
print("Scores:", scores.tolist())
print(
    f"Expected class: {test_dataset.classes[int(test_dataset[i][1])]} ({int(test_dataset[i][1])})",
)
print(
    f"Predicted class: {test_dataset.classes[scores.argmax().item()]} ({scores.argmax().item()})"
)

points = input_data[0].transpose(0, 1).numpy()
plot_points(points)

Train the segmenter:


In [ ]:
from pointnet.train_segmentation import train_segmentation

train_segmentation(
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    epochs=25,
    class_choice=0,
    outf="./pointnet/seg",
)